In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [110]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
fold1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
test_data = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='test.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [61]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [63]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
                
        # computes p_ij
        batch_pij = 1 / (1 + torch.exp(-batch_sij)) # [40, 40]
                
        # computes loss
        batch_loss = 0.5 * (1-batch_Sij) * batch_sij + torch.log(1+torch.exp(-batch_sij)) #[40, 40]
                
        # 上三角行列
        batch_loss_triu = torch.triu(batch_loss, diagonal=1)

        # 損失の平均
        num_elements = (batch_loss_triu.size(1)*batch_loss_triu.size(1)-batch_loss_triu.size(1))/2
        batch_loss_mean = torch.sum(batch_loss)/num_elements
        
        #loss_list.append(batch_loss_mean)
                
        #print("batch_loss_mean", batch_loss_mean)
        
        return batch_loss_mean
    
    def predict(self, x):
        return self.model(x)

In [64]:
# クラスのインスタンス化
model = RankNet(layers)

In [65]:
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

In [66]:
def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

In [112]:
def training_loop(n_epochs,optimizer,model,data_list,cutoff):
    # epoch 回の訓練
    for epoch in range(1, n_epochs + 1):
    
        count = 0
        test_ndcg_mean = []
        while count < 5:
            # 最初の4つを訓練に
            # 最後の1つをテストに
            train = data_list[:4]
            test = data_list[4]
            fold_train_loss = []
            fold_train_ndcg = []

            # fold loop
            for fold in train:
                loss_list = []
                ndcg_list = []

                # クエリごとの訓練
                for _,batch_ranking,label in fold:

                    batch_loss_mean = model.forward(batch_ranking=batch_ranking, label=label)
                    loss_list.append(batch_loss_mean) # クエリ一つのloss
                    
                    t = model.predict(batch_ranking)
                    r = torch.argsort(t,dim=1,descending = True)
                    ideal,_ = torch.sort(label,1,descending = True)
                    ideal = torch.unsqueeze(ideal,2).reshape(-1)
                    pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
                    # nanが含まれる場合その後の計算も全部nanになってしまったので
                    ndcg = torch.nan_to_num(nDCG(ideal,pred,cutoff))
                    ndcg_list.append(ndcg)

                fold_train_loss.append(sum(loss_list)/len(fold)) # fold1つあたりのloss
                fold_train_ndcg.append(sum(ndcg_list)/len(fold)) # fold1つあたりのndcg

            count += 1
            #　右にひとつシフト ([1,2,3,4,5] -> [5,1,2,3,4]となる)
            data_list.insert(0,data_list.pop())
        
        # fold の平均
        epoch_loss = sum(fold_train_loss)/len(fold_train_loss)
        epoch_ndcg = sum(fold_train_ndcg)/len(fold_train_ndcg)
        
        if epoch%10==0:
            print("epoch: ", epoch, "loss: ", epoch_loss, "ndcg: ", epoch_ndcg)
        
        optimizer.zero_grad()
        batch_loss_mean.backward()
        optimizer.step()

In [89]:
data_list = [fold1,fold2,fold3,fold4,fold5]

In [113]:
training_loop(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    data_list = data_list,
    cutoff = 5
    )

epoch:  10 loss:  tensor(1.4073, grad_fn=<DivBackward0>) ndcg:  tensor(0.3824)
epoch:  20 loss:  tensor(1.4157, grad_fn=<DivBackward0>) ndcg:  tensor(0.3752)
epoch:  30 loss:  tensor(1.4254, grad_fn=<DivBackward0>) ndcg:  tensor(0.3702)
epoch:  40 loss:  tensor(1.4357, grad_fn=<DivBackward0>) ndcg:  tensor(0.3679)
epoch:  50 loss:  tensor(1.4463, grad_fn=<DivBackward0>) ndcg:  tensor(0.3657)
epoch:  60 loss:  tensor(1.4566, grad_fn=<DivBackward0>) ndcg:  tensor(0.3631)
epoch:  70 loss:  tensor(1.4668, grad_fn=<DivBackward0>) ndcg:  tensor(0.3622)
epoch:  80 loss:  tensor(1.4769, grad_fn=<DivBackward0>) ndcg:  tensor(0.3612)
epoch:  90 loss:  tensor(1.4869, grad_fn=<DivBackward0>) ndcg:  tensor(0.3600)
epoch:  100 loss:  tensor(1.4967, grad_fn=<DivBackward0>) ndcg:  tensor(0.3603)


In [114]:
# test
with torch.no_grad():
    loss_sum = 0
    ndcg_sum = 0
    for _,batch_ranking,label in test_data:
        
        # computes loss
        batch_loss_mean = model.forward(batch_ranking=batch_ranking, label=label)
        loss_sum += batch_loss_mean
        
        #computes ndcg
        t = model.predict(batch_ranking)
        r = torch.argsort(t,dim=1,descending = True)
        ideal,_ = torch.sort(label,1,descending = True)
        ideal = torch.unsqueeze(ideal,2).reshape(-1)
        pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
        ndcg = torch.nan_to_num(nDCG(ideal,pred,5))
        ndcg_sum += ndcg
        
    loss_sum /= len(test_data)    
    ndcg_sum /= len(test_data)
    print("loss: ", loss_sum)
    print("ndcg: ", ndcg_sum)

loss:  tensor(1.4923)
ndcg:  tensor(0.4047)
